In [1]:
import os
import cv2
import numpy as np

In [2]:
features=[]
target=[]
for x in ['without_mask', 'with_mask']:
    ImagesNamesList=os.listdir("E:/Datasets/corona_mask" + "/" + str(x) )
    for y in ImagesNamesList:
        Imgarr=cv2.imread("E:/Datasets/corona_mask" + "/" + str(x) + "/" + y)
        try:
            Imgarr=cv2.resize(Imgarr,(100,100))
            features.append(Imgarr)
        except:
            pass
        else:
            if x=="without_mask":
                target.append(0)
            else:
                target.append(1)
        
        
        
            
    print("In Folder", x)

In Folder without_mask
In Folder with_mask


In [3]:
features=np.array(features)
target=np.array(target)


In [4]:
features.shape

(1376, 100, 100, 3)

In [5]:
target.shape

(1376,)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
features_train,features_test,target_train,target_test=train_test_split(features,target,test_size=0.2)

In [8]:
features_train.shape

(1100, 100, 100, 3)

In [9]:
target_train.shape

(1100,)

In [10]:
features_test.shape


(276, 100, 100, 3)

In [11]:
target_test.shape

(276,)

In [12]:
def preprocessing(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=image/255

    
    
    return image

In [13]:
features_train=np.array(list(map(preprocessing,features_train)))

In [14]:
features_train=features_train.reshape(1100, 100, 100,1)

In [15]:
from keras.preprocessing.image import ImageDataGenerator

In [16]:
dataGen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)

In [17]:
dataGen.fit(features_train)

In [18]:
batches=dataGen.flow(features_train,target_train,batch_size=20)

In [19]:
images,labels=next(batches)

In [20]:
from keras.utils import to_categorical

In [21]:
target_train=to_categorical(target_train)

In [22]:
target_train.shape

(1100, 2)

In [23]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential

In [24]:
model=Sequential()
model.add(Conv2D(100,(3,3),activation="relu",input_shape=(100,100,1)))
model.add(Conv2D(200,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))

In [25]:
from keras.optimizers import Adam

In [26]:
model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

In [27]:
model.fit_generator(dataGen.flow(features_train,target_train,batch_size=20),epochs=5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
55/55 [==============================] - 80s 1s/step - loss: 0.7736 - accuracy: 0.5300
Epoch 2/5
55/55 [==============================] - 80s 1s/step - loss: 0.6482 - accuracy: 0.6455
Epoch 3/5
55/55 [==============================] - 81s 1s/step - loss: 0.4624 - accuracy: 0.8000
Epoch 4/5
55/55 [==============================] - 81s 1s/step - loss: 0.4143 - accuracy: 0.8164
Epoch 5/5
55/55 [==============================] - 83s 2s/step - loss: 0.3016 - accuracy: 0.8800


In [ ]:
from keras.models import model_from_json

In [ ]:
model_json=model.to_json()
with open("verzeoAICorona.json","w") as abc:
    abc.write(model_json)
    abc.close()
model.save_weights("verzeoAICoronaWeights.h5")
print("Save the Model")

In [ ]:
json_file=open("verzeoAICorona.json","r")
loaded_model_json=json_file.read()
json_file.close()
loaded_model=model_from_json(loaded_model_json)
loaded_model.load_weights("verzeoAICoronaWeights.h5")
print("Loaded Model Sucesssfully")

In [ ]:
import cv2

In [ ]:
def getClassName(classNo):
    if   classNo == 0: return 'Without Mask'
    elif classNo == 1: return 'Mask'

In [ ]:
capt=cv2.VideoCapture(0)
capt.set(3,640)
capt.set(4,480)
capt.set(10,180)

In [ ]:
while True:
    message,image=capt.read()
    imagearr=np.asarray(image)
    face_cascade=cv2.CascadeClassifier("C:/Users/Sakthivel/haarcascade_frontalface_default.xml")
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray,1.05,3)
    for (x,y,w,h) in faces:
        detectedFace=image[x:x+w,y:y+h]
        detectedFace=cv2.resize(detectedFace,(100,100))
        detectedFace=preprocessing(detectedFace)
        detectedFace=detectedFace.reshape(1,100,100,1)
        predictions=loaded_model.predict(detectedFace)
        classIndex=loaded_model.predict_classes(imagearr)
        probabilityValue=np.amax(predictions)
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
        if probabilityValue>0.75:
            cv2.putText(image,getClassName(classIndex),(x , y-3),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.putText(image,str(int(probabilityValue * 100)) + " %",( x+w+20,y-3),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        cv2.imshow("Model Prediction",image)
        returnedValue=cv2.waitKey(1)
        if returnedValue==ord("s") or returnedValue==ord("S"):
            cv2.destroyAllWindows()
            break
